### pydeck HTML 파일을 만든 방법을 소개하는 파일입니다. 

In [2]:
import pandas as pd
import numpy as np
import pydeck as pdk

# 아파트 실거래가 데이터 불러오기
df_apt = pd.read_csv('../../../data/lab03_apart_price.csv')

# 뷰(View) 상태 정의: 지도의 초기 카메라 위치
view_state_2d = pdk.ViewState(
    longitude=126.9979,
    latitude=37.5463,
    zoom=10.5,
    pitch=0  # 지도를 바라보는 각도 (0은 수직, 높을수록 비스듬히)
)

# ScatterplotLayer 정의
layer = pdk.Layer(
    'ScatterplotLayer',
    data=df_apt,
    get_position='[경도, 위도]',
    get_color='[200, 30, 0, 160]',  # RGBA 색상
    get_radius=100,  # 각 점의 반지름 (미터 단위)
)

# Deck 객체를 렌더링
deck = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state_2d,
    map_style='light'
)

# HTML 파일로 저장
deck.to_html(
    'pydeck_apt_scatter.html',
    notebook_display=False,   # Jupyter Notebook에서 바로 렌더링하지 않음
    open_browser=False        # HTML 파일을 브라우저에서 열지 않음
)


In [3]:
import pandas as pd
import numpy as np
import pydeck as pdk

# 뷰(View) 상태 정의: 지도의 초기 카메라 위치
view_state_3d = pdk.ViewState(
    longitude=126.9979,
    latitude=37.5463,
    zoom=10.5,
    pitch=50  # 지도를 바라보는 각도 (0은 수직, 높을수록 비스듬히)
)

# HexagonLayer 정의
layer_hex = pdk.Layer(
    'HexagonLayer',
    data=df_apt,
    get_position='[경도, 위도]',
    radius=200,                 # 육각형 하나의 반지름 (미터)
    elevation_scale=4,          # 높이 배율
    elevation_range=[0, 1000],  # 높이 범위
    pickable=True,              # 마우스 클릭/호버 이벤트 활성화
    extruded=True               # 3D로 돌출시킬지 여부
)

# Deck 객체로 렌더링
deck_hex = pdk.Deck(
    layers=[layer_hex],
    initial_view_state=view_state_3d,
    map_style='light',
    tooltip={"text": "해당 지역 거래 수: {elevationValue}"}  # 간단한 툴팁
)

# HTML 파일로 저장
deck_hex.to_html(
    'pydeck_apt_hexagon.html',
    notebook_display=False,   # Jupyter Notebook에서 바로 렌더링하지 않음
    open_browser=False        # HTML 파일을 브라우저에서 열지 않음
)


### 데이터 전처리 과정 (df_apt)는 이어서 사용함

In [6]:
import pandas as pd
import numpy as np
import pydeck as pdk

# 자치구별 아파트 평균 가격 계산
df_avg_price = df_apt.groupby('구별')['가격'].mean().reset_index()

# 정규화 가격 컬럼 추가
df_avg_price['정규화가격'] = df_avg_price['가격'] / df_avg_price['가격'].max()

df_avg_price.head()


,구별,가격,정규화가격
0,강남구,14.712603,1.000000
1,강동구,5.809735,0.394882
2,강북구,3.692718,0.250990
3,강서구,5.369545,0.364962
4,관악구,4.869709,0.330989


In [8]:
import json
import pandas as pd

# GeoJSON 파일 로드
with open('../../../data/lab03_seoul_municipalities_geo.json', 'r', encoding='utf-8') as f:
    gu_json = json.load(f)

df_gu_geo = pd.DataFrame()

# 구 이름 → SIG_KOR_NM 사용
df_gu_geo["구별"] = [f["properties"]["SIG_KOR_NM"] for f in gu_json["features"]]

# 좌표 → geometry.coordinates
df_gu_geo["좌표"] = [f["geometry"]["coordinates"] for f in gu_json["features"]]

df_gu_geo.head()

,구별,좌표
0,도봉구,"[[[127.019851357, 37.700884901999984], [127.02..."
1,은평구,"[[[126.95173726300004, 37.654891147], [126.951..."
2,동대문구,"[[[127.07109526, 37.60732730400002], [127.0711..."
3,동작구,"[[[126.954724306, 37.51603819799999], [126.955..."
4,금천구,"[[[126.879936983, 37.486062359000016], [126.88..."


In [10]:
import json
import pandas as pd

# GeoJSON 파일 로드
with open('../../../data/lab03_seoul_municipalities_geo.json', 'r', encoding='utf-8') as f:
    gu_json = json.load(f)

df_gu_geo = pd.DataFrame()

# 구 이름 → SIG_KOR_NM 사용
df_gu_geo["구별"] = [f["properties"]["SIG_KOR_NM"] for f in gu_json["features"]]

# 좌표 → geometry.coordinates
df_gu_geo["좌표"] = [f["geometry"]["coordinates"] for f in gu_json["features"]]

df_gu_geo.head()

,구별,좌표
0,도봉구,"[[[127.019851357, 37.700884901999984], [127.02..."
1,은평구,"[[[126.95173726300004, 37.654891147], [126.951..."
2,동대문구,"[[[127.07109526, 37.60732730400002], [127.0711..."
3,동작구,"[[[126.954724306, 37.51603819799999], [126.955..."
4,금천구,"[[[126.879936983, 37.486062359000016], [126.88..."


In [13]:
import pandas as pd

# GeoJSON 파일을 불러와서 자치구별 경계 좌표를 추출
gu_json = pd.read_json('../../../data/lab03_seoul_municipalities_geo.json')
df_gu_geo = pd.DataFrame()

df_gu_geo['구별'] = gu_json['features'].apply(lambda row: row['properties']['SIG_KOR_NM'])
df_gu_geo['좌표'] = gu_json['features'].apply(lambda row: row['geometry']['coordinates'])

df_gu_geo.head()


,구별,좌표
0,도봉구,"[[[127.019851357, 37.700884901999984], [127.02..."
1,은평구,"[[[126.95173726300004, 37.654891147], [126.951..."
2,동대문구,"[[[127.07109526, 37.60732730400002], [127.0711..."
3,동작구,"[[[126.954724306, 37.51603819799999], [126.955..."
4,금천구,"[[[126.879936983, 37.486062359000016], [126.88..."


In [14]:
# df_avg_price와 df_gu_geo를 병합하여 구별 평균 가격과 경계 좌표를 결합
df_avg_price = df_avg_price.merge(
    df_gu_geo,
    on='구별',       # 구별 컬럼을 기준으로 병합
    how='left'      # 왼쪽 데이터프레임(df_avg_price)에 있는 모든 행을 유지
)

# 가격을 소수점 2자리의 문자열로 변환
df_avg_price['가격_문자'] = df_avg_price['가격'].apply(lambda x: f"{x:.2f}")

# 데이터프레임 확인
df_avg_price.head()


,구별,가격,정규화가격,좌표,가격_문자
0,강남구,14.712603,1.000000,"[[[127.039694597, 37.535844173999976], [127.03...",14.71
1,강동구,5.809735,0.394882,"[[[127.18162873300003, 37.55575020499998], [12...",5.81
2,강북구,3.692718,0.250990,"[[[127.00458855299996, 37.685004042], [127.005...",3.69
3,강서구,5.369545,0.364962,"[[[126.80724623599997, 37.601795434], [126.808...",5.37
4,관악구,4.869709,0.330989,"[[[126.92803474599998, 37.49502987599999], [12...",4.87


In [15]:
# PolygonLayer 정의
layer_polygon = pdk.Layer(
    'PolygonLayer',
    data=df_avg_price,
    get_polygon='좌표',               # 각 구의 경계 좌표를 지정
    opacity=0.8,                     # 폴리곤의 투명도
    get_fill_color='[0, 255*정규화가격, 0]',  # 각 데이터별 rgb 또는 rgba 값 (0~255)
    pickable=True,                   # 지도와 interactive 한 동작 on
    auto_highlight=True              # 마우스 오버(hover) 시 박스 출력
)

# 툴팁 설정
tooltip = {
    "html": "<b>{구별}</b><br/>평균가격: {가격_문자} 억원",
    "style": {"color": "white", "backgroundColor": "steelblue"}
}

# Deck 객체로 렌더링
deck_polygon = pdk.Deck(
    layers=[layer_polygon],
    initial_view_state=view_state_2d,  # 초기 뷰 상태
    map_style='dark',                  # 지도 스타일
    tooltip=tooltip                    # 툴팁 설정
)

# Deck 객체를 HTML 파일로 저장
deck_polygon.to_html(
    'pydeck_apt_polygon_2d.html',
    notebook_display=False,   # Jupyter Notebook에서 바로 렌더링하지 않음
    open_browser=False        # HTML 파일을 브라우저에서 열지 않음
)

In [ ]:
# PolygonLayer 정의
layer_polygon = pdk.Layer(
    'PolygonLayer',
    data=df_avg_price,
    get_polygon='좌표',                 # '좌표' 열에서 경계 좌표를 가져옴
    stroked=True,                      # 폴리곤의 외곽선을 표시
    extruded=True,                     # 3D로 돌출
    get_elevation='정규화가격*3000',   # 높이 설정: 정규화된 가격
    get_fill_color='[0, 255*정규화가격, 255]',  # 각 데이터별 RGB 값 (0~255)
    pickable=True,                     # 지도와 interactive한 동작 on
    auto_highlight=True                # 마우스 오버(hover) 시 박스 출력
)

# 툴팁 설정
tooltip = {
    "html": "<b>{구별}</b><br/>평균가격: {가격_문자} 억원",
    "style": {"color": "white", "backgroundColor": "steelblue"}
}

# Deck 객체로 렌더링
deck_polygon = pdk.Deck(
    layers=[layer_polygon],
    initial_view_state=view_state_3d,  # 초기 뷰 상태
    map_style='dark',                  # 지도 스타일
    tooltip=tooltip                    # 툴팁 설정
)

# Deck 객체를 HTML 파일로 저장
deck_polygon.to_html(
    'pydeck_apt_polygon_3d.html',
    notebook_display=False,   # Jupyter Notebook에서 바로 렌더링하지 않음
    open_browser=False        # HTML 파일을 브라우저에서 열지 않음
)
